In [1]:
# !pip install PytorchCML

In [1]:
import sys
sys.path.append("../../src/")

from itertools import product

from PytorchCML import losses, models, samplers, evaluators, trainers
import torch
from torch import nn, optim
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
movielens = pd.read_csv(
  'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
  sep='\t', header=None, index_col=None,
)
movielens.columns = ["user_id", "item_id", "rating", "timestamp"]
movielens.user_id -= 1
movielens.item_id -= 1
movielens.rating = (movielens >= 4).astype(int)
n_user = movielens.user_id.nunique()
n_item = movielens.item_id.nunique()

train, test = train_test_split(movielens)


# all user item pairs
df_all = pd.DataFrame(
    [[u, i] for u,i in product(range(n_user), range(n_item))],
    columns=["user_id", "item_id"]
)

# frag train pairs
df_all = pd.merge(
    df_all, 
    train[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
)

# remove train pairs
test = pd.merge(
    df_all[df_all.rating.isna()][["user_id", "item_id"]], 
    test[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
).fillna(0)

# numpy array
train_set = train[train.rating == 1][["user_id", "item_id"]].values
test_set = test[["user_id", "item_id", "rating"]].values


## Strict Negative

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=True)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(torch.LongTensor(test_set).to(device), score_function_dict, ks=[3,5])
trainer = trainers.CMLTrainer(model, optimizer, criterion, sampler)


In [4]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:15<00:00, 59.91it/s]


In [5]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.015895,0.028455,0.001643,0.015187,0.033086,0.002452,0,NaN
0,0.057471,0.091110,0.004146,0.063293,0.107319,0.008038,10,1212.098991
0,0.254217,0.345263,0.029643,0.251258,0.355607,0.048304,20,738.998521


## Not Strict Negative

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(torch.LongTensor(test_set).to(device), score_function_dict, ks=[3,5])
trainer = trainers.CMLTrainer(model, optimizer, criterion, sampler)


In [8]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:25<00:00, 36.62it/s]


In [9]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.019532,0.034376,0.002140,0.020022,0.040712,0.003958,0,NaN
0,0.059776,0.093319,0.005672,0.064944,0.111475,0.009510,10,1345.827259
0,0.255193,0.344291,0.030209,0.251102,0.359304,0.050676,20,925.398547
